# Purpose


This notebook demonstrates the data pipeline from raw tables to analytical datasets. At the end of this activity, train & test data sets are created from raw data.



## Imports

In [1]:
from pprint import pprint
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.options.mode.use_inf_as_na = True


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
from numba import NumbaDeprecationWarning

warnings.filterwarnings('ignore', message="The default value of regex will change from True to False in a future version.",
                        category=FutureWarning)

warnings.filterwarnings("ignore", category=NumbaDeprecationWarning)

In [ ]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, get_package_path,
    display_as_tabs, string_cleaning, merge_info, initialize_environment,
    list_datasets, load_dataset, save_dataset
)
import ta_lib.eda.api as eda

In [5]:
initialize_environment(debug=False, hide_warnings=True)

## Utility functions

# 1. Initialization

In [6]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
pprint(list_datasets(context))

housing_df = load_dataset(context, 'raw/housing')

['/raw/housing',
 '/cleaned/housing',
 '/train/housing/features',
 '/train/housing/target',
 '/test/housing/features',
 '/test/housing/target',
 '/score/housing/output']


# 2. Data cleaning and consolidation

## 2.1 Clean individual tables 

### Cleaning housing dataset:

- Remove duplicate rows
- Convert data types (e.g., ensure numeric types for price, area, etc.)
- Handle missing values appropriately
- Generate additional features like age_of_house, price_per_sqft


In [7]:
housing_df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [8]:
housing_df_clean = (
    housing_df
    .copy()
    .drop_duplicates()
    .replace({'': np.NaN})
    .convert_dtypes()
)

# Generate derived features
housing_df_clean['rooms_per_household'] = housing_df_clean['total_rooms'] / housing_df_clean['households']
housing_df_clean['bedrooms_per_room'] = housing_df_clean['total_bedrooms'] / housing_df_clean['total_rooms']
housing_df_clean['population_per_household'] = housing_df_clean['population'] / housing_df_clean['households']

### NOTE

It's always a good idea to save cleaned tabular data using a storage format that supports the following 

1. preserves the type information
2. language agnostic storage format
3. Supports compression
4. Supports customizing storage to optimize different data access patterns

For larger datasets, the last two points become crucial.

`Parquet` is one such file format that is very popular for storing tabular data. It has some nice properties:
- Similar to pickles & RDS datasets, but compatible with all languages
- Preserves the datatypes
- Compresses the data and reduces the filesize
- Good library support in Python and other languages
- As a columnar storage we can efficiently read fewer columns
- It also supports chunking data by groups of columns (for instance, by dates or a particular value of a key column) that makes loading subsets of the data fast.

In [9]:
save_dataset(context, housing_df_clean, 'cleaned/housing')

In [10]:
save_dataset(context, orders_df_clean, 'cleaned/orders')

# 3. Generate Train, Validation and Test datasets



- We split the data into train, test (optionally, also a validation dataset)
- In this example, we are binning the target into 10 quantiles and then use a Stratified Shuffle to split the data.
- See sklearn documentation on the various available splitters
- https://scikit-learn.org/stable/modules/classes.html#splitter-classes
- This will go into production code (training only)

In [10]:
splitter = train_test_split(housing_df_clean, test_size=0.2, random_state=context.random_seed)
housing_train, housing_test = splitter

target_col = "median_house_value"

train_X = housing_train.drop(columns=[target_col])
train_y = housing_train[target_col]
save_dataset(context, train_X, 'train/housing/features')
save_dataset(context, train_y, 'train/housing/target')

In [11]:
test_X = housing_test.drop(columns=[target_col])
test_y = housing_test[target_col]
save_dataset(context, test_X, 'test/housing/features')
save_dataset(context, test_y, 'test/housing/target')